In [1]:
using JuMP
using Gurobi, Ipopt
using PyPlot
using LinearAlgebra
using CSV
using DataFrames
using NonlinearSolve

In [5]:
using Pkg
Pkg.add("DataStructures")

    Updating registry at `C:\Users\nevin\.julia\registries\General.toml`
   Resolving package versions...
    Updating `C:\Users\nevin\.julia\environments\v1.8\Project.toml`
  [864edb3b] + DataStructures v0.18.13
  No Changes to `C:\Users\nevin\.julia\environments\v1.8\Manifest.toml`


In [2]:
# read data
l1 = DataFrame(CSV.File("data\\top_2.csv"))
lm = DataFrame(CSV.File("data\\middle_2.csv"))
l3  = DataFrame(CSV.File("data\\bottom_2.csv"))

Row,1.447213595499957872e+00,7.763932022500210639e-01
,Float64,Float64
1,1.47256,0.830615
2,1.49827,0.88373
3,1.52434,0.935739
4,1.55077,0.986641
5,1.57754,1.03644
6,1.60465,1.08513
7,1.63209,1.13271
8,1.65986,1.17918
9,1.68794,1.22455


In [ ]:
N = 50
lx = l1[!,1][1:N]
ly = l1[!,2][1:N]

rx = l3[!,1][1:N]
ry = l3[!,2][1:N]

mx = lm[!,1][1:N]
my = lm[!,2][1:N]

cx1 = (lx + mx)/2
cy1 = (ly + my)/2

cx2 =  (rx + mx)/2
cy2 =  (ry + my)/2

ox = cx2[20]
oy = cy2[20]

plot(lx,ly)
plot(rx,ry)
plot(mx,my)
plot(cx1,cy1)
plot(cx2, cy2)
scatter(ox,oy)

In [ ]:
cx1[1:19]

In [ ]:
cx = cx2[1:19]
append!(cx, cx1[20:end])
cy = cy2[1:19]
append!(cy, cy1[20:end])
plot(lx,ly)
plot(rx,ry)
plot(mx,my)
scatter(cx,cy)

In [ ]:
# function E_cur(i)
#     i = trunc(Int, i)
#     if i == 1
#         v1 = X[i,:]
#         v2 = X[i+1,:] - X[i,:]
#         norm_v1 = v1'*v1
#         norm_v2 = v2'*v2
#         dTheta = acos( dot(v1,v2)/ (norm_v1*norm_v2))
#         deltaX = X[i,:]'*X[i,:]
#         return dTheta/deltaX
#     elseif i == N
#         v1 = X[i,:] - X[i-1,:]
#         v2 = -X[i,:]
#         norm_v1 = v1'*v1
#         norm_v2 = v2'*v2
#         v1v2 = dot(v1,v2)
#         #dTheta = acos( dot(v1,v2)/ (norm_v1*norm_v2))
#         @NLexpression(m, dTheta,  acos(v1v2/ (norm_v1*norm_v2)))
#         dx_norm =  (X[i,:] - X[i-1,:])'*(X[i,:] - X[i-1,:]) 
#         @NLexpression(m, deltaX, dx_norm)
#         return dTheta/deltaX
#     else
#         v1 = X[i,:] - X[i-1,:]
#         v2 = X[i+1,:] - X[i,:]
#         norm_v1 = v1'*v1
#         norm_v2 = v2'*v2
        
#         dTheta = acos( dot(v1,v2)/ (norm_v1*norm_v2))
#         deltaX = (X[i,:] - X[i-1,:])'*((X[i,:] - X[i-1,:]))
#         return dTheta/deltaX       
#     end
# end

In [ ]:
function E_cur_i(xi,yi,xprev,yprev,xnext,ynext)
    v1x = xi - xprev
    v1y = yi - yprev
    
    v2x = xnext - xi
    v2y = ynext - yi

    v1v2 = v1x*v2x + v1y*v2y
    v1norm = sqrt(v1x^2 + v1y^2)
    v2norm = sqrt(v2x^2 + v2y^2)
    
    deltaTheta = acos(v1v2/(v1norm*v2norm))
    deltaX = v1norm
    
    return deltaTheta/deltaX
end

In [ ]:
function E_dev_i(x,y,cx,cy)
    dx = (x - cx)^2
    dy = (y - cy)^2
    #println(dx + dy)
    return dx + dy
end

In [ ]:
function E_obs_i(xi,yi,ox,oy)
    thresh = 0.5
    dxo = (xi - ox)^2
    dyo = (yi - oy)^2
    dobsnorm = sqrt(dxo + dyo)
    
    E_obs = 0.0
    if dobsnorm > 0 && dobsnorm < thresh
        E_obs = ((1/dobsnorm) - 1)^2
        #println(dobsnorm)
    end
    
    return E_obs
end

In [ ]:
function E_i(xi,yi,xprev,yprev,xnext,ynext,cx,cy,ox,oy)
    
#     # curvature term
#     v1x = xi - xprev
#     v1y = yi - yprev
    
#     v2x = xnext - xi
#     v2y = ynext - yi

#     v1v2 = v1x*v2x + v1y*v2y
#     v1norm = sqrt(v1x^2 + v1y^2)
#     v2norm = sqrt(v2x^2 + v2y^2)
    
#     deltaTheta = acos(v1v2/(v1norm*v2norm))
#     deltaX = v1norm
    
#     E_cur = deltaTheta/deltaX
    
    # Deviation term
    dx = (xi - cx)^2
    dy = (yi - cy)^2
    
    E_dev = (dx + dy)
    
#     # Obstacle term
#     thresh = 0.5
#     dxo = (xi - ox)^2
#     dyo = (yi - oy)^2
#     dobsnorm = sqrt(dxo + dyo)
    
#     E_obs = 0
#     if dobsnorm < thresh
#         E_obs = ((1/dobsnorm) - 1)^2
#     end
    #println("Hello", E_dev)
    return E_dev
end

In [ ]:
function ∇E_i()

In [ ]:
m = Model(Ipopt.Optimizer)
register(m, :E_cur_i, 6, E_cur_i, autodiff=true)
register(m, :E_dev_i, 4, E_dev_i, autodiff=true)
register(m, :E_obs_i, 4, E_obs_i, autodiff=true)
#register(m, :E_i, 10, E_i, autodiff=true)
w_cur = 1
w_dev = 1
w_obs = 1
@variable(m,x[1:N])
@variable(m, y[1:N])
set_start_value.(x, cx)
set_start_value.(y, cy)


#delta_theta = [ (x[i] - x[i+1]) for i = 1:99]
#@NLexpression(m, E_cur, sum(E_cur_i(x[i],y[i], x[i-1],y[i-1], x[i+1],y[i+1]) for i in 2:99))
@NLexpression(m, E_dev, sum(E_dev_i(x[i],y[i], cx[i], cy[i]) for  i in 1:N))
@NLexpression(m, E_obs, sum(E_obs_i(x[i],y[i],ox,oy)  for i in 1:N))
#@NLexpression(m, E_cur, sum(  acos( dot((X[i,:] - X[i-1,:]), (X[i+1,:] - X[i,:]))/(  ((X[i,:] - X[i-1,:])'*(X[i,:] - X[i-1,:]))*((X[i+1,:] - X[i,:])'*(X[i+1,:] - X[i,:])) )) /  ((X[i,:] - X[i-1,:])'*((X[i,:] - X[i-1,:])))    for i in 2:99) )
@NLobjective(m, Min, E_dev + E_obs)

#@NLobjective(m, Min, sum(E_i(x[i],y[i], x[i-1],y[i-1], x[i+1],y[i+1],cx[i],cy[i], ox,oy) for i in 2:99 ))
#print(m)

optimize!(m)

In [ ]:
xopt = value.(x)
yopt = value.(y)

In [ ]:
#plot(lx,ly)
#plot(rx,ry)
#plot(cx,cy)
plot(xopt,yopt)

In [7]:
mutable struct PIDController
    Kp::Float64
    Ki::Float64
    Kd::Float64
    setpoint::Float64
    last_error::Float64
    integral::Float64
end

function PIDController(Kp::Float64, Ki::Float64, Kd::Float64, setpoint::Float64)
    return PIDController(Kp, Ki, Kd, setpoint, 0.0, 0.0)
end

function update!(pid::PIDController, measured_value::Float64, dt::Float64)
    error = pid.setpoint - measured_value
    pid.integral += error * dt
    derivative = (error - pid.last_error) / dt
    output = pid.Kp * error + pid.Ki * pid.integral + pid.Kd * derivative
    pid.last_error = error

    return output
end


update! (generic function with 1 method)

In [8]:
function simulate_system(pid_controller::PIDController, plant, timesteps, dt)
    system_state = 0.0
    control_input_history = []
    system_state_history = []

    for t in 1:timesteps
        control_input = update!(pid_controller, system_state, dt)
        push!(control_input_history, control_input)

        system_state = plant(system_state, control_input, dt)
        push!(system_state_history, system_state)
    end

    return system_state_history, control_input_history
end

# Define a simple plant model
function plant(state, control_input, dt)
    return state + control_input * dt
end

# Set up PID controller
Kp = 1.0
Ki = 0.5
Kd = 0.1
setpoint = 5.0
pid_controller = PIDController(Kp, Ki, Kd, setpoint)

# Simulate the system
timesteps = 100
dt = 0.1
system_state_history, control_input_history = simulate_system(pid_controller, plant, timesteps, dt)

# Print the results
println("System state history: ", system_state_history)
println("Control input history: ", control_input_history)


System state history: Any[1.0250000000000001, 1.364875, 1.7574506250000002, 2.121711371875, 2.466768975015625, 2.791907287340547, 3.097564161077844, 3.384115670494785, 3.6520019870495934, 3.901706181299886, 4.133749637449101, 4.348686073606258, 4.5470961377788734, 4.729582352043787, 4.8967644181127605, 5.049274870303867, 5.18775506431213, 5.312851489416291, 5.425212391453364, 5.525484693636172, 5.614311202117944, 5.692328083111054, 5.760162598338163, 5.818431085627471, 5.8677371715534905, 5.908670203165469, 5.941803886031827, 5.9676951160559515, 5.986882992781608, 5.999888002200637, 6.0072113573974235, 6.009334485709768, 6.0067186514507735, 5.999804703617558, 5.989012938406999, 5.974743066763195, 5.95737427759328, 5.937265387705001, 5.914755069938861, 5.890162151387427, 5.863785974012729, 5.835906810387764, 5.806786327698383, 5.776668093545889, 5.745778117487227, 5.714325422637612, 5.682502642038865, 5.650486634864712, 5.618439117891191, 5.586507308005504, 5.554824571859574, 5.52351107